# Parking mandates

## Hypothesis

The off-street parking mandate may be requiring more space dedicated to parking than is necessary.

In [1]:
import pandas as pd
import geopandas as gpd

# data fetching
from data.connect_db import get_db

con = get_db()

## Fetch American Community Survey data for Waltham census tracts

In [2]:
to_keep = ["DP04_0057E", "DP04_0058E", "DP04_0059E", "DP04_0060E", "DP04_0061E"]

query = f"""
select
    *
from
    data_profiles_census
"""

data_profiles_df = pd.read_sql(query, con, index_col='tract', columns=["tract"] + to_keep)

data_profiles_df.index.name = "tract"

data_profiles_df.head()

,DP03_0052E,DP03_0052PE,DP03_0053E,DP03_0053PE,DP03_0054E,DP03_0054PE,DP03_0055E,DP03_0055PE,DP03_0056E,DP03_0056PE,...,DP03_0060PE,DP03_0061E,DP03_0061PE,DP03_0062E,DP04_0134E,DP04_0057E,DP04_0058E,DP04_0059E,DP04_0060E,DP04_0061E
tract,,,,,,,,,,,,,,,,,,,,,
3681.01,132.0,5.7,124.0,5.3,34.0,1.5,234.0,10.1,56.0,2.4,...,11.4,558.0,24.0,118358.0,2705.0,2323.0,194.0,927.0,959.0,243.0
3681.02,18.0,0.9,19.0,1.0,109.0,5.5,87.0,4.4,97.0,4.9,...,18.3,679.0,34.3,153727.0,1853.0,1978.0,68.0,525.0,735.0,650.0
3682,8.0,0.5,38.0,2.5,30.0,2.0,108.0,7.2,63.0,4.2,...,15.0,414.0,27.5,127984.0,2593.0,1506.0,70.0,564.0,600.0,272.0
3683,19.0,0.8,180.0,7.3,159.0,6.4,247.0,10.0,232.0,9.4,...,8.0,290.0,11.7,75969.0,1457.0,2480.0,320.0,951.0,802.0,407.0
3684,72.0,3.2,42.0,1.8,162.0,7.1,27.0,1.2,176.0,7.7,...,11.2,408.0,17.9,99174.0,2051.0,2285.0,108.0,1003.0,1013.0,161.0


In [3]:
subject_df = pd.read_sql("select * from subject_census", con, index_col='tract')

subject_df.index.name = "tract"

subject_df.head()

,S0101_C01_026E,S0101_C01_001E,S0101_C01_002E,S0101_C01_003E,S0101_C01_004E,S0101_C01_005E,S0101_C01_006E,S0101_C01_007E,S0101_C01_008E,S0101_C01_009E,...,S0102_C02_005E,S0102_C02_006E,S0102_C02_007E,S0102_C02_008E,S0102_C02_009E,S0102_C02_010E,S0102_C02_011E,S0102_C02_012E,S0102_C02_013E,S0102_C02_014E
tract,,,,,,,,,,,,,,,,,,,,,
3681.01,4545.0,5524.0,299.0,364.0,298.0,18.0,516.0,574.0,496.0,467.0,...,None,None,None,None,None,None,None,None,None,None
3681.02,4696.0,5367.0,160.0,177.0,165.0,264.0,835.0,309.0,441.0,242.0,...,None,None,None,None,None,None,None,None,None,None
3682,3078.0,3673.0,174.0,131.0,157.0,221.0,185.0,182.0,246.0,211.0,...,None,None,None,None,None,None,None,None,None,None
3683,4599.0,5241.0,327.0,136.0,125.0,166.0,629.0,831.0,538.0,429.0,...,None,None,None,None,None,None,None,None,None,None
3684,7242.0,8109.0,346.0,141.0,301.0,1674.0,1746.0,405.0,481.0,581.0,...,None,None,None,None,None,None,None,None,None,None


## Calculate number of adults per car

The usual assumption is that every adult in a household unit 'needs' a car, and any new construction should provide that amount of parking,
so incoming residents do not use on-street parking for vehicle storage.

This is set as ['2 per dwelling unit'](https://ecode360.com/26938091) for family dwellings.

This is assuming the typical 'nuclear' family with two adults who are doing the driving. If all adults are driving, one would expect that the number
of adults = the number of cars, so this ratio should come out to 1.

I have yet to find data that gets the number of cars in a census tract, so I have decided to calculate this from the survey questions as follows:

`n_cars = 1 * households with one car + 2 * households with two cars + 3 * households with 3 or more cars`

*Assumption* - in most cases, the number of cars in the '3 or more category' is going to tend to be closer to 3 than other values, for most homes.
If this assumption is wrong, `n_cars` will be lower than the actual amount.

In [4]:
# Estimate!!Total!!Total population!!SELECTED AGE CATEGORIES!!18 years and over
total_adults = subject_df["S0101_C01_026E"]

# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!No vehicles available
# "DP04_0058E",
# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!1 vehicle available
# "DP04_0059E",
# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!2 vehicles available
# "DP04_0060E",
# # Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available
# "DP04_0061E"
parking_mandates_df = pd.DataFrame(columns=["total_vehicles", "total_adults", "adults_per_vehicle"])

parking_mandates_df["total_adults"] = subject_df["S0101_C01_026E"]


# ASSUMPTION - more than 3 vehicles will be rare, so assume 3 per household unit for that bucket
parking_mandates_df["total_vehicles"] = (1 * data_profiles_df["DP04_0059E"] + 2 * data_profiles_df["DP04_0060E"] + 3 * data_profiles_df["DP04_0061E"]).values

parking_mandates_df["adults_per_vehicle"] = parking_mandates_df["total_adults"] / parking_mandates_df["total_vehicles"]

print(parking_mandates_df)

         total_vehicles  total_adults  adults_per_vehicle
tract                                                    
3681.01          3574.0        4545.0            1.271684
3681.02          3945.0        4696.0            1.190368
3682             2580.0        3078.0            1.193023
3683             3776.0        4599.0            1.217956
3684             3512.0        7242.0            2.062073
3685             2017.0        2846.0            1.411006
3686             3915.0        5213.0            1.331545
3687             1664.0        2257.0            1.356370
3688             3814.0        4778.0            1.252753
3689.01          4703.0        5263.0            1.119073
3689.02          1914.0        2781.0            1.452978
3690               49.0        4165.0           85.000000
3691             3673.0        4183.0            1.138851


## Comments

Most tracts come in at more than one adult per vehicle, except for tract 3690 and 3684.

3690 is mostly Bentley University, so we would expect many adults who are non-driving students.

3684 mostly overlaps ward 7, which is dominated by single family homes. It appears that many adults are sharing vehicles, but
this tract also contains Brandeis University. 

*Assumption* - It's not possible to tell if the all of the [3,031 18-24 year olds in tertiary eduation](https://data.census.gov/map?g=0400000US25_1400000US25017368101,25017368102,25017368200,25017368300,25017368400,25017368500,25017368600,25017368700,25017368800,25017368901,25017368902,25017369000,25017369100&tid=ACSST5Y2020.S1401&cid=S1401_C01_030E&layer=VT_2020_140_00_PY_D1&mode=thematic&loc=42.3818,-71.2223,z12.3362) in this tract are going to Brandeis, but assuming they are, the ratio would come out to

`(7091 - 3031) / 3527 = ~1.15 adults / vehicle`

## Source

This data comes from the 5-year American Community Survey estimate from 2020. If anything, I would assume that the number
of vehicles would drop somewhat due to more adults no longer needing to commute to an office.

You can look at the uncombined data on the tracts [here](https://data.census.gov/map?q=DP04&g=1400000US25017368101,25017368102,25017368200,25017368300,25017368400,25017368500,25017368600,25017368700,25017368800,25017368901,25017368902,25017369000,25017369100&tid=ACSDP5Y2020.DP04&cid=DP04_0060PE&layer=VT_2020_140_00_PY_D1&mode=thematic&loc=42.3628,-71.2427,z13.6751). Change the `Variable(s)` field
to see how each tract fits in each category.